## TODO

Dla każdej kategorii zbudować tf-idf matrix

In [39]:
import pandas as pd
import json 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

train_df = pd.read_csv("data/train.csv")
meta_df = pd.read_csv("data/item_metadata_filtered.csv")
with open("data/id_mappings.json") as f:
    id_map = json.load(f)

asin_to_id = {asin: int(item_id) for asin, item_id in id_map["item_mapping"].items()}
item_mapping_df = pd.DataFrame(list(asin_to_id.items()), columns=["parent_asin", "item_id"])
meta_df = pd.merge(meta_df, item_mapping_df, on="parent_asin", how="left")
# train_df["main_category"] = item_mapping_df[train_df["item_id"] == item_mapping_df["item_id"]]["main_category"]
train_df = train_df.merge(meta_df[["item_id", "main_category"]], on="item_id", how="left")


# Preprocess metadata
meta_df["title"] = meta_df["title"].fillna("")
meta_df["store"] = meta_df["store"].fillna("")
meta_df["description"] = meta_df["description"].fillna("")
meta_df["average_rating"] = meta_df["average_rating"].fillna("")
meta_df["price"] = meta_df["price"].fillna("")
meta_df["image_urls"] = meta_df["image_urls"].fillna("[]")
meta_df["main_category"] = meta_df["main_category"].fillna("")

# Index by ASIN for lookup
meta_df = meta_df.set_index("parent_asin")

# --- TF-IDF ---


tfidf_matrix_dictionary = {}
for category in meta_df["main_category"].unique():
    # Filter ASINs for current category
    category_df = meta_df[meta_df["main_category"] == category]
    
    texts = [
        row["title"] + " " + row["store"] + " " + row["description"]
        for _, row in category_df.iterrows()
    ]
    
    vectorizer = TfidfVectorizer(max_features=10000)
    tfidf_matrix_dictionary[category] = vectorizer.fit_transform(texts)
    
    print(f"Processed category: {category}, TF-IDF shape: {tfidf_matrix_dictionary[category].shape}")
tfidf_matrix_dictionary

Processed category: All Beauty, TF-IDF shape: (17952, 10000)
Processed category: Premium Beauty, TF-IDF shape: (111, 751)
Processed category: Health & Personal Care, TF-IDF shape: (10716, 10000)
Processed category: Appstore for Android, TF-IDF shape: (43463, 10000)
Processed category: Software, TF-IDF shape: (5471, 10000)
Processed category: , TF-IDF shape: (113, 4449)
Processed category: Gift Cards, TF-IDF shape: (4, 17)
Processed category: Computers, TF-IDF shape: (2, 56)
Processed category: Home Audio & Theater, TF-IDF shape: (1, 11)


{'All Beauty': <17952x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 456065 stored elements in Compressed Sparse Row format>,
 'Premium Beauty': <111x751 sparse matrix of type '<class 'numpy.float64'>'
 	with 1412 stored elements in Compressed Sparse Row format>,
 'Health & Personal Care': <10716x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 303313 stored elements in Compressed Sparse Row format>,
 'Appstore for Android': <43463x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 4076735 stored elements in Compressed Sparse Row format>,
 'Software': <5471x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 715216 stored elements in Compressed Sparse Row format>,
 '': <113x4449 sparse matrix of type '<class 'numpy.float64'>'
 	with 15271 stored elements in Compressed Sparse Row format>,
 'Gift Cards': <4x17 sparse matrix of type '<class 'numpy.float64'>'
 	with 60 stored elements in Compressed Sparse Row format>,
 'Computers': <2x56 sparse 

In [40]:
category_df = {}
for category in meta_df["main_category"].unique():
    category_df[category] = meta_df[meta_df["main_category"] == category]
    category_df[category].drop(columns="main_category", axis=1)
    
category_df["All Beauty"].head()

,main_category,title,average_rating,rating_number,price,store,features,description,images,categories,image_count,has_images,image_urls,category,item_id
parent_asin,,,,,,,,,,,,,,,
B07WFSQXL5,All Beauty,PPY Eyelash Growth Serum – Natural Ingredients...,3.9,114.0,,PPY,[],[],{'hi_res': array(['https://m.media-amazon.com/...,[],27,True,['https://m.media-amazon.com/images/I/615N6fkc...,All_Beauty,66457
B08BV6F6BC,All Beauty,Wixar Natural Sea Moss Soap - (2 PACK) - Laven...,4.4,41.0,,WIXAR NATURALS,[],[],{'hi_res': array(['https://m.media-amazon.com/...,[],21,True,['https://m.media-amazon.com/images/I/81ugegqe...,All_Beauty,71310
B07Z818MLY,All Beauty,7 Packs Deep Wave Crochet Hair 22 Inch Deep wa...,3.4,10.0,,Yun Mei Hair,[],[],{'hi_res': array(['https://m.media-amazon.com/...,[],21,True,['https://m.media-amazon.com/images/I/71aVcpK8...,All_Beauty,67591
B071DY8Z4B,All Beauty,BEWAVE Hair Brush Sponge Twist With Comb Hair ...,4.2,24.0,,BEWAVE,[],[],{'hi_res': array(['https://m.media-amazon.com/...,[],17,True,['https://m.media-amazon.com/images/I/61or2jYp...,All_Beauty,52585
B0BTLTVR1X,All Beauty,"Zydeco Chop Chop Cajun Seasoning Base, 8 Ounce...",4.7,21.0,,BORELTH,"['All Natural blend of Dehydrated Onion, Dehyd...",['Zydeco Chop Chop is a blend of Dehydrated On...,{'hi_res': array(['https://m.media-amazon.com/...,[],3,True,['https://m.media-amazon.com/images/I/71707mY6...,All_Beauty,77706


In [ ]:
item_id_to_index_dictionary = {
    category: {
        item_id: idx for idx, item_id in enumerate(df["item_id"].values)
    }
    for category, df in category_df.items()
}
def build_user_profile(user_id):
    user_ratings = train_df[train_df["user_id"] == user_id]
    categories = user_ratings["main_category"].unique()

    profile_parts = []
    tmp_df = pd.DataFrame()  # optional: collect for debugging or analysis

    for category in categories:
        category_ratings = user_ratings[user_ratings["main_category"] == category]
        item_ids = category_ratings["item_id"].values
        ratings = category_ratings["rating"].values

        # Mapping: item_id -> index in TF-IDF matrix
        id_to_index = item_id_to_index_dictionary.get(category, {})
        indices = [id_to_index[item_id] for item_id in item_ids if item_id in id_to_index]
        filtered_ratings = [r for item_id, r in zip(item_ids, ratings) if item_id in id_to_index]

        if not indices:
            continue  # no items in TF-IDF matrix for this category

        # Get the item vectors
        tfidf_vectors = tfidf_matrix_dictionary[category][indices]

        # Store for analysis (optional)
        tmp_df = pd.concat([
            tmp_df,
            pd.DataFrame({
                "category": category,
                "item_id": item_ids,
                "rating": ratings
            })
        ])

        # Weight the vectors by user ratings
        weighted = tfidf_vectors.multiply(np.array(filtered_ratings)[:, None])
        profile_parts.append(weighted.mean(axis=0))

    if not profile_parts:
        return np.zeros((1, tfidf_matrix_dictionary[categories[0]].shape[1])), tfidf_vectors

    # Combine all category profiles (average)
    full_profile = np.mean(profile_parts, axis=0)
    return full_profile, tfidf_vectors

def recommend_for_user(user_id, top_k=10):
    user_items = train_df[train_df["user_id"] == user_id]["item_id"]
    categories = meta_df[meta_df["item_id"].isin(user_items)]["main_category"].unique()

    profile, tfidf_matrix = build_user_profile(user_id)
    scores = cosine_similarity(profile, tfidf_matrix).ravel()

    seen_items = set(train_df.loc[train_df["user_id"] == user_id, "item_id"])

    n_items = len(scores)
    top_n = min(top_k * 5, n_items)  # safe top-N

    ranked_indices = np.argpartition(-scores, range(top_n))[:top_n]
    top_items = sorted(ranked_indices, key=lambda i: -scores[i])

    recs = [str(i) for i in top_items if i not in seen_items][:top_k]
    return ' '.join(recs)

def solve_and_save(min, max, num):
    # for key, df in category_df:

    df_final = pd.DataFrame({
        "user_id": train_df["user_id"].unique()[min:max],
    })
    # df_final["predictions"] = df_final["user_id"].apply(recommend_for_user, args=(train_df["main_category"]))
    df_final["predictions"] = df_final["user_id"].apply(
        lambda user_id: recommend_for_user(user_id)
    )

    name = f"user_predictions{num}.csv"
    print(name, flush=True)
    df_final.to_csv(name, index=False)



In [102]:
def recommend_for_user(user_id, top_k=10):
    user_items = train_df[train_df["user_id"] == user_id]["item_id"]
    categories = meta_df[meta_df["item_id"].isin(user_items)]["main_category"].unique()

    profile, tfidf_matrix = build_user_profile(user_id)
    scores = cosine_similarity(profile, tfidf_matrix).ravel()

    seen_items = set(train_df.loc[train_df["user_id"] == user_id, "item_id"])

    n_items = len(scores)
    top_n = min(top_k * 5, n_items)  # safe top-N

    ranked_indices = np.argpartition(-scores, range(top_n))[:top_n]
    top_items = sorted(ranked_indices, key=lambda i: -scores[i])

    recs = [str(i) for i in top_items if i not in seen_items][:top_k]
    return ' '.join(recs)

def solve_and_save(min, max, num):
    # for key, df in category_df:

    df_final = pd.DataFrame({
        "user_id": train_df["user_id"].unique()[min:max],
    })
    # df_final["predictions"] = df_final["user_id"].apply(recommend_for_user, args=(train_df["main_category"]))
    df_final["predictions"] = df_final["user_id"].apply(
        lambda user_id: recommend_for_user(user_id)
    )

    name = f"user_predictions{num}.csv"
    print(name, flush=True)
    df_final.to_csv(name, index=False)



In [ ]:
''' item_id_to_index = {item_id: idx for idx, item_id in enumerate(meta_df["item_id"])}

def recommend_for_user(user_id, top_k=10):
    user_items = train_df[train_df["user_id"] == user_id]["item_id"]
    user_categories = meta_df[meta_df["item_id"].isin(user_items)]["main_category"]
    category = user_categories.mode().iloc[0]

    profile, tfidf_matrix = build_user_profile(user_id)
    scores = cosine_similarity(profile, tfidf_matrix).ravel()

    seen_items = set(train_df.loc[train_df["user_id"] == user_id, "item_id"])

    # Use numpy argsort only once and avoid recomputing reverse sort
    ranked_indices = np.argpartition(-scores, range(top_k * 5))[:top_k * 5]  # over-sample in case of seen items
    top_items = sorted(ranked_indices, key=lambda i: -scores[i])

    
    recs = [str(i) for i in top_items if i not in seen_items][:top_k]
    
    return ' '.join(recs)

 '''

In [ ]:
''' def build_user_profile(user_id, category):
    
    # user_ratings = train_df[train_df["main_category"] == category]
    user_ratings = train_df[traind_df["user_id"] == user_id]
    categories = user_ratings["main_category"].unique()

    indicies = user_ratings["item_id"].values
    ratings = user_ratings["rating"].values
    
    tmp_df = pd.DataFrame()  
    ratings = [fidi_matrix_dictionary[cat] for cat in categories]
    item_vectors = tfidf_matrix_dictionary[category][indices]

    # indices = user_ratings["item_id"].values
    # ratings = user_ratings["rating"].values
    # item_vectors = tfidf_matrix_dictionary[category][indices]


    weighted = item_vectors.multiply(ratings[:, None])
    profile = weighted.mean(axis=0)
    return np.asarray(profile)
 '''



In [95]:
def solve_and_save(min, max, num):
    # for key, df in category_df:

    df_final = pd.DataFrame({
        "user_id": train_df["user_id"].unique()[min:max],
    })
    # df_final["predictions"] = df_final["user_id"].apply(recommend_for_user, args=(train_df["main_category"]))
    df_final["predictions"] = df_final["user_id"].apply(
        lambda user_id: recommend_for_user(user_id)
    )

    name = f"user_predictions{num}.csv"
    print(name, flush=True)
    df_final.to_csv(name, index=False)



In [ ]:
from multiprocessing import Process

for i, min_idx in enumerate(range(0, 20000, 20000)):
    max_idx = min(min_idx + 20000, 868218)
    p = Process(target=solve_and_save, args=(min_idx, max_idx, i))
    processes.append(p)
    p.start()


/home/pete/Coding/Python/DSC_task/env/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
Process Process-28:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-106-7b2f1b219ce3>", line 77, in solve_and_save
    lambda user_id: recommend_for_user(user_id)
  File "/home/pete/Coding/Python/DSC_task/env/lib/python3.6/site-packages/pandas/core/series.py", line 4213, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas

In [104]:

from multiprocessing import Process
def solve_and_save(min, max, num):
    df_final = pd.DataFrame({
        "user_id": train_df["user_id"].unique()[min:max],
    })
    df_final["predictions"] = df_final["user_id"].apply(recommend_for_user)
    name = f"user_predictions{num}.csv"
    print(name, flush=True)
    df_final.to_csv(name, index=False)

processes = []

solve_and_save(0, 20000, 10)

''' for i, min_idx in enumerate(range(0, 20000, 20000)):
    max_idx = min(min_idx + 20000, 868218)
    p = Process(target=solve_and_save, args=(min_idx, max_idx, i))
    processes.append(p)
    p.start()
 '''
# Optional: Wait for all to finish
# for p in processes:
    # p.join()

/home/pete/Coding/Python/DSC_task/env/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


ValueError: could not broadcast input array from shape (751) into shape (1)

In [ ]:
from tqdm import tqdm

user_predictions = []
count = 5
for i in tqdm(train_df["user_id"].unique()):
    scores = recommend_for_user(i)
    str_scores = ", ".join([str(s[0]) for s in scores])
    user_predictions.append({"user_id": i, "predictions": str_scores})

df_out = pd.DataFrame(user_predictions)
df_out

  0%|          | 0/868218 [00:00<?, ?it/s]

  0%|          | 2/868218 [00:00<18:38:27, 12.94it/s]

[0.03262974 0.26607072 0.23708686 ... 0.         0.04430078 0.02940371]
[0.01633298 0.17650924 0.15808626 ... 0.01631689 0.02221824 0.01469989]
[0.0096187  0.17099702 0.15624592 ... 0.         0.03250344 0.01602827]


  0%|          | 6/868218 [00:00<16:45:57, 14.38it/s]

[0.01320169 0.27133719 0.24773484 ... 0.01114314 0.03782189 0.02583157]
[0.0086747  0.17611851 0.16643569 ... 0.         0.02141305 0.02617324]
[0.01198627 0.14670529 0.14372686 ... 0.00807202 0.02046561 0.01483715]


...

  0%|          | 712/868218 [00:51<17:43:17, 13.60it/s]

[0.01528507 0.19149532 0.16148864 ... 0.         0.02447321 0.01268692]
[0.01609081 0.20652127 0.19161526 ... 0.         0.01311288 0.03875603]
[0.00677785 0.04946763 0.04612257 ... 0.         0.00154067 0.00852269]


  0%|          | 715/868218 [00:51<17:17:38, 13.93it/s]

[0.01890863 0.22463936 0.21100187 ... 0.02567065 0.02006733 0.01619037]
[0.00757628 0.06521151 0.07484496 ... 0.         0.01044185 0.00900498]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import tqdm

# Step 1: Build user profile
def build_user_profile(user_id):
    user_ratings = train_df[train_df["user_id"] == user_id]
    indices = user_ratings["item_id"].values
    ratings = user_ratings["rating"].values
    item_vectors = tfidf_matrix[indices]
    weighted = item_vectors.multiply(ratings[:, None])
    profile = weighted.mean(axis=0)
    return np.asarray(profile)

# Step 2: Build user predictions
user_predictions = []


for user_id in tqdm(train_df["user_id"].unique()):
    profile = build_user_profile(user_id)
    scores = cosine_similarity(profile, tfidf_matrix).flatten()

    # Optionally filter out items already rated by the user
    seen_items = set(train_df[train_df["user_id"] == user_id]["item_id"])
    predictions = [str(score) for i, score in enumerate(scores) if i not in seen_items]

    user_predictions.append({
        "user_id": user_id,
        "predictions": " ".join(predictions)
    })

# Step 3: Save to DataFrame
df_output = pd.DataFrame(user_predictions)
print("Done")

# Step 4: Export to CSV
df_output.to_csv("user_predictions.csv", index=False)


NameError: name 'tqdm' is not defined